In [1]:
import numpy as np
import scipy
import scipy.special as sp
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as axes3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
from Legendre import *
from clebschgordan import *
from wigner3j import *
from vsh import *
from spharm import *
from hankel import *
from domain import *
from LaguerreGauss import *
from d_jmp import *
from C_jlp import *
from matplotlib import rc
from mie_coefs import *

## Define Standard Units
fsize = 22
tsize = 15
tdir = 'in'
major = 5
minor = 3
style = 'default'

params = {
    'figure.figsize': (15,12),
    'savefig.dpi': 75,
    'text.usetex': False,
    'font.size': fsize,
    'legend.fontsize': tsize,
    'legend.title_fontsize': tsize,
    'mathtext.fontset' : 'stix',
    'font.family' : 'STIXGeneral',    
    'axes.labelsize':15,
    'axes.titlesize':20,
    'lines.linewidth':2.5,
    'axes.grid': False,
    'axes.labelweight':'bold',
    'legend.loc': 'upper right',
    'xtick.labelsize':'x-small',
    'ytick.labelsize':'x-small',
}
plt.rcParams.update(params)

In [2]:
def d_jmp(j, m, p, Theta):
    lnCoef = 0.5 * (sp.gammaln(j - m + 1) + sp.gammaln(j + m + 1) - sp.gammaln(j + p + 1) - sp.gammaln(j - p + 1))
    cosFac = np.cos(Theta / 2) ** (m + p)
    sinFac = (-np.sin(Theta / 2)) ** (m - p)
    
    n = j - m
    alpha = m - p
    beta = m + p
    hyp = sp.eval_jacobi(n, alpha, beta, np.cos(Theta))
    
    d_jmp = np.exp(lnCoef) * cosFac * sinFac * hyp
    
    return d_jmp

\begin{align*}
    D_{m^{\prime}m}^j(\alpha\beta\gamma)=e^{-im^{\prime}\alpha}d_{m^{\prime}m}^j(\beta)e^{-im\gamma}
\end{align*}

In [4]:
def D_jmp(j, m, p, alpha, beta, gamma):
    fac1 = np.exp(-1j * m * alpha)
    fac2 = d_jmp(j, m, p, beta)
    fac3 = np.exp(-1j * p * gamma)
    return fac1 * fac2 * fac3

In [ ]:
def compute_C_off(j_max, mz, p, k, d, D_matrix_func, clebsch_func, C_con):
    """
    Compute C^off as a vector for j in range(0, j_max).
    
    Parameters:
    - j_max: Maximum j value
    - mz, p: Indices
    - k, d: Wavevector and displacement parameter
    - D_matrix_func: Function to compute Wigner D-matrix elements
    - clebsch_func: Function to compute Clebsch-Gordan coefficients
    - C_con: Known C^on values
    
    Returns:
    - C_off: Array of C^off values for each j
    """
    C_off = np.zeros(j_max+1, dtype=np.complex128)
    
    for j in range(j_max+1):
        sum_Ln = 0
        for j_prime in range(j_max+1):
            for L in range(j + j_prime + 1):  # Summation over L
                J_L = spherical_jn(L, k*d)  # Compute Bessel function
                factor = (2*L + 1) * (-1j)**L * J_L
                
                for n in range(-min(j, j_prime), min(j, j_prime)+1):
                    CG_1 = clebsch_func(j, n, L, 0, j_prime, n)
                    CG_2 = clebsch_func(j, p, L, 0, j_prime, p)
                    
                    sum_Ln += factor * CG_1 * CG_2
            
            # Compute Wigner D-matrix contributions
            D_jm = D_matrix_func(j, mz, j_prime)
            D_jn = D_matrix_func(j_prime, n, mz)
            
            C_off[j] += D_jm * D_jn * C_con[j_prime, mz, p] * sum_Ln

    return C_off
